In [1]:
import pandas as pd
from dse_do_utils import DataManager, OptimizationEngine,ScenarioManager
from docplex.mp.model import Model
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

from docplex import __version__
__version__

'2.25.236'

Borrowed from "Ashok D. Belegundu; Tirupathi R. Chandrupatla, Optimization Concepts and Applications in Engineering, 2011 "

VLSI (very large-scale integration) is a process used to build electronic components
such as microprocessors and memory chips comprising millions of transistors.
The first stage of the VLSI design process typically produces a set of
indivisible rectangular blocks called cells. In a second stage, interconnection
information is used to determine the relative placements of these cells. In a
third stage, implementations are selected for the various cells with the goal of
optimizing the total area, which is related to cost of the chips. It is the third
stage, floor plan optimization, for which we give a simple example below.

We are given three rectangular cells. Dimensions of C1 is 5 × 10, C2 can be
chosen as 3 × 8, 2 × 12, or 6 × 4, and C3 can be chosen 5 × 8 or 8 × 5.

 Relative ordering of the cells must satisfy the following vertical and horizontal ordering:

Let ($w_i$ , $h_i$ ), $i = 1, 2, 3$, denote the width and height of cell i, and ($x_i$, $y_i$) denote the coordinates of the left bottom corner of cell $i$. $W$ and $H$ are the sizes of bounding rectangle, where the area supposed to be maximized. 

![](vlsi.png)

$y_1 + h_1 ≤ H$

$y_3 + h_3 ≤ y_2$

$y_2 + h_2 ≤ H$

$x_1 + w_1 ≤ x_2$

$x_1 + w_1 ≤ x_3$

$x_2 + w_2 ≤ W$

$x_3 + w_3 ≤ W$

In [2]:
mdl = Model(name="vlsi")

In [3]:
h1 = mdl.continuous_var(lb = 2, ub = 10, name = 'h1')
h2 = mdl.continuous_var(lb = 2,  ub = 10, name = 'h2')
h3 = mdl.continuous_var(lb = 2,  ub = 10, name = 'h3')

w1 = mdl.continuous_var(lb = 2,  ub = 10, name = 'w1')
w2 = mdl.continuous_var(lb = 2,  ub = 10, name = 'w2')
w3 = mdl.continuous_var(lb = 2,  ub = 10, name = 'w3')

H = mdl.continuous_var(lb = 2, ub = 15,  name = 'H')
W = mdl.continuous_var(lb = 2, ub = 15,  name = 'W')

x1 = mdl.continuous_var(lb = 0,  ub = 15, name = 'x1')
x2 = mdl.continuous_var(lb = 0,  ub = 15, name = 'x2')
x3 = mdl.continuous_var(lb = 0,  ub = 15, name = 'x3')

y1 = mdl.continuous_var(lb = 0,  ub = 15, name = 'y1')
y2 = mdl.continuous_var(lb = 0,  ub = 15, name = 'y2')
y3 = mdl.continuous_var(lb = 0,  ub = 15, name = 'y3')



In [4]:
mdl.add_constraint(y1 + h1 <= H)
mdl.add_constraint(y3 + h3 <= y2)
mdl.add_constraint(y2 + h2 <= H)
mdl.add_constraint(x1 + w1 <= x2)
mdl.add_constraint(x1 + w1 <= x3)
mdl.add_constraint(x2 + w2 <= W)
mdl.add_constraint(x3 + w3 <= W)

docplex.mp.LinearConstraint[](w3+x3,LE,W)

Introducing the binary (or 0/1) variables $d_{i j}$ to implement discrete selection

In [5]:
d21 = mdl.binary_var( name = 'd21')
d22 = mdl.binary_var(name = 'd22')
d23 = mdl.binary_var(name = 'd23')

d31 = mdl.binary_var( name = 'd31')
d32 = mdl.binary_var( name = 'd32')

In [6]:
options = [['C1', 5 , 10], ['C2',3,8],['C2',2,12],['C2',6,4],['C3',5,8],['C3',8,5]]
options = pd.DataFrame(options, columns = ['cell','h','w'])
options['cases'] = [0, d21, d22, d23, d31,d32]
options

,cell,h,w,cases
0,C1,5,10,0
1,C2,3,8,d21
2,C2,2,12,d22
3,C2,6,4,d23
4,C3,5,8,d31
5,C3,8,5,d32


In [7]:
mdl.add_constraint(8*d21 + 12 * d22 + 4*d23 == w2)
mdl.add_constraint(3*d21 + 2 * d22 + 6*d23 == h2)

mdl.add_constraint(5*d31 + 8 * d32  == w3)
mdl.add_constraint(8*d31 + 5 * d32  == h3)

mdl.add_constraint(d21 +  d22 + d23  == 1)
mdl.add_constraint(d31 +  d32  == 1)


docplex.mp.LinearConstraint[](d31+d32,EQ,1)

# !!!For testing only. add constraint for known optimal solution!!!

mdl.add_constraint(x1   == 0)  # added for testing only
mdl.add_constraint(y1   == 0)  # added for testing only
mdl.add_constraint(w1   == 5)  # added for testing only
mdl.add_constraint(h1   == 10)  # added for testing only

mdl.add_constraint(x2   == 5)  # added for testing only
mdl.add_constraint(y2   == 7)  # added for testing only
mdl.add_constraint(w2   == 8)  # added for testing only
mdl.add_constraint(h2   == 3)  # added for testing only

mdl.add_constraint(x3   == 5)  # added for testing only
mdl.add_constraint(y3   == 0)  # added for testing only
mdl.add_constraint(w3   == 8)  # added for testing only
mdl.add_constraint(h3   == 5)  # added for testing only

mdl.add_constraint(d32   == 1)  # added for testing only
mdl.add_constraint(d21   == 1)  # added for testing only4
mdl.add_constraint(W   == 13)  # added for testing only4
mdl.add_constraint(H   == 10)  # added for testing only4

In [8]:
#https://www.ibm.com/docs/en/icos/22.1.1?topic=parameters-linearization-switch-qp-miqp
#https://www.ibm.com/docs/en/icos/22.1.1?topic=smippqt-miqp-mixed-integer-programs-quadratic-terms-in-objective-function

mdl.parameters.optimalitytarget = 3
mdl.parameters.dettimelimit = 120
mdl.parameters.preprocessing.qtolin = 1
#mdl.parameters.emphasis.numerical =1

mdl.print_information()

Model: vlsi
 - number of variables: 19
   - binary=5, integer=0, continuous=14
 - number of constraints: 13
   - linear=13
 - parameters:
     parameters.dettimelimit = 120.00000000000000
     parameters.optimalitytarget = 3
     parameters.preprocessing.qtolin = 1
 - objective: none
 - problem type is: MILP


In [9]:
area = W * H
mdl.add_kpi(area   , "area")
mdl.minimize(area)


assert mdl.solve(log_output=True), "Solve failed" 
mdl.report()

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_OptimalityTarget                        3
CPXPARAM_Preprocessing_QToLin                    1
CPXPARAM_DetTimeLimit                            120
Found incumbent of value 110.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIQP Presolve eliminated 4 rows and 8 columns.
MIQP Presolve modified 2 coefficients.
Aggregator did 6 substitutions.
Reduced MIQP has 4 rows, 6 columns, and 9 nonzeros.
Reduced MIQP has 2 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 2 nonzeros.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.01 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf

In [10]:
print(f"Solve Status = {mdl.solve_details.status}")

Solve Status = integer optimal solution


In [11]:
mdl.print_solution()

objective: 80.000
status: OPTIMAL_SOLUTION(2)
  h1=2.000
  h2=3.000
  h3=5.000
  w1=2.000
  w2=8.000
  w3=8.000
  H=8.000
  W=10.000
  x2=2.000
  x3=2.000
  y2=5.000
  d21=1
  d32=1


In [12]:
mdl.report_kpis()

*  KPI: area = 80.000


In [13]:
#mdl.export_as_mps("vlsi1.mps")